In [ ]:

from qspmodel import *
from folfiri_qspmodel import *
import pandas as pd
import csv
import os
import scipy as sp

if not os.path.exists('Results/'):
    os.makedirs('Results/Var_sens_par/')
else:
    if not os.path.exists('Results/Var_sens_par/'):
        os.makedirs('Results/Var_sens_par/')
        

# some global parameters
clusters=5 #number of clusters

number_of_cycles=12
cycle_time=14
#injection_time=2 # days

# treatment starts at time 0
start_treatment = 0

T=3500
t=np.linspace(0, T, 35001)

# pre-defined parameters for treatment deltas, cell_steady, cell_extreme, drug_ratio, drug_eff, 5FU_killrate
drug_deltas=[71.3, 1.45, 2.56]
cell_steady_ind=[5, 6, 13]
cell_extreme_ind=[5, 6, 12]
drug_levels=np.array([770, 300, 725])/cycle_time #per day; median drug dosages from patient data
drug_min=np.array([598, 208, 75])/cycle_time #per day; min drug dosages from patient data
drug_ratio=drug_levels/drug_min
drug_eff=[0.2, 0.4, 0.05]
FU_killrate=100

# create parameters for step function 
infusion_indices=np.concatenate([15*np.ones(number_of_cycles), 16*np.ones(number_of_cycles), 17*np.ones(number_of_cycles)]).astype(int)
FUinfusion=np.array([2/24, 2]) 
IRinfusion=np.array([0, 1/24]) 
LVinfusion=np.array([1/24, 2/24]) 
infusion_intervals=np.concatenate([[start_treatment+n*cycle_time+FUinfusion for n in range(number_of_cycles)],
                                    [start_treatment+n*cycle_time+IRinfusion for n in range(number_of_cycles)],
                                    [start_treatment+n*cycle_time+LVinfusion for n in range(number_of_cycles)]])
infusion_values=np.concatenate([(cycle_time/(FUinfusion[1]-FUinfusion[0]))*drug_deltas[0]*np.ones(number_of_cycles),
                                (cycle_time/(IRinfusion[1]-IRinfusion[0]))*drug_deltas[1]*np.ones(number_of_cycles),
                                (cycle_time/(LVinfusion[1]-LVinfusion[0]))*drug_deltas[2]*np.ones(number_of_cycles)])

#other parameters
nvar=Colon_5fu_Functions().nvar # number of variables
nparam=Colon_5fu_Functions().nparam # number of parameters

# infusion function
r= step_vector(nvar, indices=infusion_indices, intervals=infusion_intervals, values=infusion_values)
jump_indicator=lambda t, x: np.prod(t-infusion_intervals.flatten())

# reading mean-based cell data for sensitivity analysis
clustercells=pd.read_csv('input/Large_Tumor_cell_data.csv').to_numpy()
extremevals=np.array([2.67309712e+04, 1.23113649e+04, 1.91073391e+04, 7.21023854e+03,
                              3.41730382e+03, 4.32746936e+03, 2.31601361e+04, 3.24717521e+05,
                              1.62358760e+05, 1.05768591e+03, 1.39828734e+04, 2.46968870e+04,
                              1.02214368e+02, 1.63407223e+05])
#Computations of dynamics
print('Starting dynamics computations')

#initial conditions at steady-state
IC= np.ones((clusters,nvar))
IC[:,-3:]=0

#choose which parameters to vary by position
par_position = ([8,3]) # delta_C5fuLV and delta_5fuM 

#constants to multiply to the parameter
cons = ([0.1,0.5,1,2,3,4,5])

for p in range(len(par_position)):
    pos = par_position[p]
    folder_name = Colon_5fu_Functions().parameter_names[pos][1:] 
    #create folder to store data corresponding to the parameter
    if not os.path.exists('Results/Var_sens_par/'+folder_name+'/'):
     os.makedirs('Results/Var_sens_par/'+folder_name+'/') 
    
    print('Varying parameter '+ folder_name)
    for cluster in range(clusters):
        print('Starting computations for cluster '+str(cluster+1)+' of 5') 
        for c in range(len(cons)):
            
            QSP0=QSP.from_cell_data(clustercells[cluster])
            qspcore=Colon_5fu_Functions(parameters=QSP0.par, inject=False)
            QSP_=QSP(parameters=qspcore.parameters_from_assumptions([drug_deltas, (clustercells[cluster,cell_steady_ind]/extremevals[cell_extreme_ind]), drug_ratio, drug_eff, FU_killrate]), qspcore=qspcore)
            par_base=np.copy(QSP_.par)
            new_par=np.copy(par_base)
            # set new parameter value 
            new_par[pos]=new_par[pos]*cons[c]
            QSP_.set_parameters(new_par)

            filename='cluster-'+str(cluster+1)+'-results-'+str(c)+'-'

            print(' Parameters set. Computing the solution')
            sol = sp.integrate.solve_ivp((lambda t, x: QSP_(t, x)+r(t)), (0, T), IC[cluster], t_eval=t, jac=(lambda t, x: QSP_.Ju(t,x)), events=jump_indicator)
            u=sol.y.T

            wr=np.empty((t.size, nvar+1))
            wr[:,0]=t
            wr[:,1:]=u
            c=csv.writer(open('Results/Var_sens_par/'+folder_name+'/'+filename+'dat.csv',"w"))  
            c.writerow(['time']+QSP_.variable_names())
            c.writerows(wr)
            del c




Starting dynamics computations
Varying parameter delta_{CFULV}
Starting computations for cluster 1 of 5
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
Starting computations for cluster 2 of 5
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
Starting computations for cluster 3 of 5
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters set. Computing the solution
 Parameters se